# 6.838, Shape Analysis:  Homework 4

All homeworks will be graded out of 100 points.
<hr/>
Again we provide some code at the top for displaying meshes with per-vertex color.  We also provide code for the mass matrix and cotangent Laplacian for piecewise-linear FEM on a triangulated surface for your convenience.

In [1]:
import numpy as np
import trimesh
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import matplotlib.cm as cm
from scipy.sparse import spdiags
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs

def map_z2color(zval, colormap, vmin, vmax):
    t=(zval-vmin)/float((vmax-vmin)); R, G, B, alpha=colormap(t)
    return 'rgb('+'{:d}'.format(int(R*255+0.5))+','+'{:d}'.format(int(G*255+0.5))+','+'{:d}'.format(int(B*255+0.5))+')'   

def tri_indices(simplices):
    return ([triplet[c] for triplet in simplices] for c in range(3))

def plotly_trisurf(x, y, z, colors, simplices, colormap=cm.RdBu, plot_edges=False):
    
    points3D=np.vstack((x,y,z)).T
    tri_vertices=map(lambda index: points3D[index], simplices)
    
    ncolors = (colors-np.min(colors))/(np.max(colors)-np.min(colors))
    
    I,J,K=tri_indices(simplices)
    triangles=Mesh3d(x=x,y=y,z=z,
                     intensity=ncolors,
                     colorscale='Viridis',
                     i=I,j=J,k=K,name='',
                     showscale=True,
                     colorbar=ColorBar(tickmode='array', 
                                       tickvals=[np.min(z), np.max(z)], 
                                       ticktext=['{:.3f}'.format(np.min(colors)), 
                                                 '{:.3f}'.format(np.max(colors))]))
    
    if plot_edges is False: # the triangle sides are not plotted
        return Data([triangles])
    else:
        lists_coord=[[[T[k%3][c] for k in range(4)]+[ None] for T in tri_vertices] for c in range(3)]
        Xe, Ye, Ze=[reduce(lambda x,y: x+y, lists_coord[k]) for k in range(3)]
        lines=Scatter3d(x=Xe,y=Ye,z=Ze,mode='lines',line=Line(color='rgb(50,50,50)', width=1.5))
        return Data([triangles, lines])

def textured_mesh(mesh, per_vertex_signal, filename):
    x = mesh.vertices.transpose()[0]; y = mesh.vertices.transpose()[1]; z = mesh.vertices.transpose()[2];
    data1 = plotly_trisurf(x, y, z, per_vertex_signal, mesh.faces, colormap=cm.RdBu,plot_edges=True)
    axis = dict(showbackground=True,backgroundcolor="rgb(230, 230,230)",gridcolor="rgb(255, 255, 255)",zerolinecolor="rgb(255, 255, 255)")
    layout = Layout(width=800, height=800,scene=Scene(xaxis=XAxis(axis),yaxis=YAxis(axis),zaxis=ZAxis(axis),
                                                     aspectmode='data'))
    fig1 = Figure(data=data1, layout=layout)
    iplot(fig1, filename=filename)
    
def row_norms(mtx):
    return np.sum(np.abs(mtx)**2,axis=-1)**.5

def barycentric_areas(X,T): # I'll go ahead and do part (d) here
    vv = []
    I = T[:,0]; J = T[:,1]; K = T[:,2];
    vv.append( X[I,:] ); vv.append( X[J,:] ); vv.append( X[K,:] )

    # Triangle areas
    nn = np.cross(vv[1]-vv[0],vv[2]-vv[0])
    triangleAreas = .5*row_norms(nn)

    # Angle deficits (integrated curvature) and barycentric areas
    barycentricAreas = np.zeros(nv)
    for i in range(0, 3):
        barycentricAreas = barycentricAreas + np.bincount(T[:,i],triangleAreas/3)
    
    return barycentricAreas

def mass_matrix(X,T):
    nv = X.shape[0]
    return spdiags(barycentric_areas(X,T),0,nv,nv)

def cot_laplacian(X,T):
    nv = X.shape[0]
    nt = T.shape[0]
    
    vv = []
    I = T[:,0]; J = T[:,1]; K = T[:,2];
    vv.append( X[I,:] ); vv.append( X[J,:] ); vv.append( X[K,:] )

    # Triangle areas
    nn = np.cross(vv[1]-vv[0],vv[2]-vv[0])
    triangleAreas = .5*row_norms(nn)

    # Angle deficits (integrated curvature) and barycentric areas
    innerCotangents = np.zeros((nt,3))
    for i in range(0, 3):
        e1 = vv[(i+1)%3]-vv[i]
        e2 = vv[(i+2)%3]-vv[i]
        innerCotangents[:,i] = np.sum(np.multiply(e1,e2),axis=-1)/(2*triangleAreas) # dot product over cross product
    
    L = (csr_matrix((innerCotangents[:,2],(I,J)),shape=(nv,nv)) +
         csr_matrix((innerCotangents[:,0],(J,K)),shape=(nv,nv)) +
         csr_matrix((innerCotangents[:,1],(K,I)),shape=(nv,nv))
        )
    
    L = L+L.transpose()
    rowSums = np.sum(L,axis=-1).transpose()
    L = L - spdiags(rowSums,0,nv,nv)
    return -.5*L

<b>Problem 1:  Exterior calculus</b>

<b>a.  Hodge star on $\mathbb{R}^3$ (15 points).</b>  Take $\{dx^1,dx^2,dx^3\}$ to be the standard basis for 1-forms over $\mathbb{R}^3$.  Recall we can write a basis for 2-forms over $\mathbb{R}^3$ as $\{dx^1\wedge dx^2,dx^1\wedge dx^3,dx^2\wedge dx^3\}$, and that the set of 3-forms is spanned by a single differential form $dx^1\wedge dx^2\wedge dx^3$.  Given differential 1-forms $\omega=\sum_{i=1}^3 \omega_i dx^i$ and $\nu=\sum_{i=1}^3 \nu_i dx^i$, where $\omega,\nu:\mathbb{R}^3\rightarrow\mathbb{R}$, write expressions in these bases (and using partial derivatives of the component functions $\omega_i,\nu_i$) for:
<ul>
<li> $d\omega$
<li> $\langle \omega,\nu\rangle$
<li> $\star\omega$
</ul>

<b>Answer:</b>


<b>b.  Practice with exterior calculus notation (15 points extra credit).</b>  Suppose $f:\mathbb{R}^3\rightarrow\mathbb{R}$ is a function and that $F:\mathbb{R}^3\rightarrow\mathbb{R}^3$ is a vector field.  Verify the following identities:
<ul>
<li> $\nabla f = (df)^\sharp$
<li> $\nabla\cdot F = \star d \star(F^\flat)$
<li> $\nabla\times F = (\star d(F^\flat))^\sharp$
</ul>
If you choose not to answer this extra credit problem, you can still use the relationships above in future problems on this problem set.

<b>Answer:</b>  


<b>c. Circumcenter distance (10 points).</b>  Suppose triangle $XYZ$ has circumcenter $B$ and that triangle $ZYW$ has circumcenter $C$; notice these two triangles share an edge $YZ$.  Derive an expression for the ratio $\frac{|BC|}{|YZ|}$ in terms of the interior angles of these two triangles; you may assume they are both acute triangles.

<p><b>Hint:</b> Unsurprisingly for 6.838, the answer will involve only $\cot\angle YXZ$ and $\cot\angle YWZ$.</p>
<p><b>Hint 2:</b> You may wish to show that $\angle BZY=\frac{\pi}{2}-\angle YXZ$.  Drawing lots of pictures helps, and take a look at <a href="https://en.wikipedia.org/wiki/Circumscribed_circle">this article</a>.</p>

<b>Answer:</b>  

<b>d.  Hodge star of 1-forms (5 points).</b>  Suppose we are implementing DEC on a manifold triangle mesh without boundary.  Use your answer to (b) to propose a Hodge star operator $\star_{11}^{PD}\in\mathbb{R}^{|E|\times|E|}$ from discrete primal 1-forms to discrete dual 1-forms.

<p><b>Hint:</b>  The matrix should be diagonal.</p>

<b>Answer:</b>  

<b>e.  Exterior Laplacian (5 points).</b>  Use your answer in (a) to an expression for the Laplacian on functions (0-forms) in terms of $\star$'s and $d$'s.

<b>Answer:</b> 

<b>f.  Connection to finite element method (5 points).</b>  Show that after discretization using DEC, your answer to (d) can be factored as $M^{-1}L$ where $M$ is a lumped mass matrix and $L$ is the cotangent Laplacian.

<p><b>Note:</b>  This problem shows that "all roads lead to Rome."  The Laplacian from DEC is identical to the Laplacian from FEM.</p>

<b>Answer:</b> 

<b>Problem 2:  Heat method</b>

<b>a.  Heat diffusion (10 points).</b>  The heat equation on a smooth surface is given by $\frac{\partial u}{\partial t}=-\Delta u$, where $\Delta$ is the Laplacian with sign chosen to be positive definite.  Suppose we have a triangle mesh $(V,T)$ and an initial distribution of heat $u_0\in\mathbb{R}^{|V|}$.  Explain how to use the finite element method (for spatial discretization) and implicit time stepping (for time discretization) to approximate $u_t\in\mathbb{R}^{|V|}$, the distribution of heat after diffusing for time $t$.

<p><b>Note:</b>  For notation, assume your mesh has cotangent Laplacian $L$ and mass matrix $M$; divide $t$ into $k$ time steps.</p>

<b>Answer:</b>


<b>b.  Implementation (10 points).</b>  Implement your method from (a) below.

In [2]:
from scipy.sparse.linalg import spsolve

def diffuse_heat(X,T,u0,k,t):
    u = u0
    
    # DO k STEPS OF DIFFUSION TO TOTAL TIME t HERE 
    
    return u

# Test on a simple mesh
mesh = trimesh.load_mesh('cup.off')
X = mesh.vertices; # each row is the position of a vertex
I,J,K=tri_indices(mesh.faces)
T = np.column_stack((I,J,K)) # rows are (i,j,k) indices of triangle vertices
nv = X.shape[0] # number of vertices
nt = T.shape[0] #number of triangles

# Put a pin prick of heat on the surface
u0 = np.zeros((nv,1))
u0[0] = 1 # point a point of heat at vertex 0

# Do 5 iterations of diffusion to time t=.1
k = 5
t = .1
u = diffuse_heat(X,T,u0,k,t)

# Display
textured_mesh(mesh,u,'diffusion')

<b>c.  Approximating geodesic distances (10 points).</b>  Take $x,y\in S$ to be two points on a surface $S$.  <a href="https://www.cs.cmu.edu/~kmcrane/Projects/GeodesicsInHeat/paper.pdf">Varadhan's Theorem</a> states that the geodesic distance from $x$ to $y$ can be recovered as
$$d(x,y)=\lim_{t\rightarrow0} \sqrt{-4t\log k_{t,x}(y)},$$
where $k$ is the <i>heat kernel</i> measuring the amount of heat traveled from $x$ to $y$ in time $t$.

Suppose $v\in V$, and take some small but positive $0<\varepsilon\ll1$.  Using your answer to (a), define $u_\varepsilon$ to be an approximation of heat diffusion for time $\varepsilon$, where $u_0$ is all zero except with a 1 on vertex $v$.  Propose a "quick and dirty" way to approximate the geodesic distance from $v$ to all other vertices $w\in V$ in terms of $u_\varepsilon$, and implement this technique.

Discuss the advantages and drawbacks of your geodesic distance approximation from a numerical perspective.

<b>Text answer:</b> 

In [3]:
centerVertex = 0 # Approximate geodesic distances from this vertex

dist = np.zeros((X.shape[0],1)) # REPLACE WITH GEODESIC DISTANCE APPROXIMATION

textured_mesh(mesh,dist,'distance')

<b>Problem 3:  Heat kernel map</b>  In this problem, you will implement the <a href="http://www.lix.polytechnique.fr/~maks/papers/hks.pdf">heat kernel signature</a> for points on a surface.

<b>a.  Laplacian eigenfunctions (5 points).</b>  Write code for approximating the first $k$ eigenfunctions/eigenvalues of the Laplacian of a triangulated surface (eigenvalues closest to zero).  Don't forget to use a mass matrix!

In [4]:
from scipy.sparse.linalg import eigsh

def laplacian_spectrum(X,T,k):
    # REPLACE WITH CODE TO COMPUTE EIGENVALUES AND EIGENVECTORS
    return np.zeros((X.shape[0],1)),np.zeros((X.shape[0],k))
    
mesh = trimesh.load_mesh('human_coarse.off')

# For convenience, collect matrices of vertices (nv x 3) and triangles (nt x 3)
X = mesh.vertices; # each row is the position of a vertex
I,J,K=tri_indices(mesh.faces)
T = np.column_stack((I,J,K)) # rows are (i,j,k) indices of triangle vertices
nv = X.shape[0] # number of vertices
nt = T.shape[0] #number of triangles

# Compute 10 eigenfunctions and display eigenfunction number 4 for fun
k = 10
vals,vecs = laplacian_spectrum(X,T,k)
textured_mesh(mesh,vecs[:,3],'eigenfunction') # display an eigenfunction -- play with this to see them all!

<b>b.  Heat kernel signature per-vertex (10 points).</b>  The heat kernel signature is a function of $t>0$ at each point $x$ on a surface:
$$\mathrm{HKS}(x,t):=\sum_{i=0}^\infty e^{-\lambda_i t}\phi_i(x)^2,$$
where $\phi_i$ is the $i$-th Laplace-Beltrami eigenfunction normalized to square-integrate to 1.  Implement an approximation of this function per-vertex below.

In [5]:
from numpy.matlib import repmat
from math import log

def HKS(eigenvalues,eigenvectors,nSamples):
    neig = eigenvalues.shape[0]
    tmin = 4*log(10)/eigenvalues[neig-1]
    tmax = 4*log(10)/eigenvalues[1]
    t = np.logspace(log(tmin,10),log(tmax,10),nSamples)
    
    # COMPUTE HKS HERE
    hks = np.zeros((eigenvectors.shape[0],nSamples))
    
    return hks

<b>c.  HKS of key points (10 points).</b>  The code below plots  the HKS as a function of $t$ for the two hands and two feet of the human model.  Discuss the pattern you see.  Is it what you expect?

In [6]:
import sys

nSamples = 500
k = 300 # number of eigenfunctions used to approximate HKS

# Compute HKS for first human
mesh = trimesh.load_mesh('human_coarse.off');sys.stdout.flush()
X = mesh.vertices; # each row is the position of a vertex
I,J,K=tri_indices(mesh.faces)
T = np.column_stack((I,J,K)) # rows are (i,j,k) indices of triangle vertices
vals,vecs = laplacian_spectrum(X,T,k)
hks = HKS(vals,vecs,nSamples)

In [7]:
hand1 = 259
hand2 = 135
knee1 = 232
knee2 = 257

import plotly.offline as py
import plotly.graph_objs as go

x = np.linspace(1,nSamples,nSamples)

trace0 = go.Scatter(x=x,y=hks[hand1,:],name='Hand 1')
trace1 = go.Scatter(x=x,y=hks[hand2,:],name='Hand 2')
trace2 = go.Scatter(x=x,y=hks[knee1,:],name='Knee 1')
trace3 = go.Scatter(x=x,y=hks[knee2,:],name='Knee 2')

data = [trace0,trace1,trace2,trace3]
layout = go.Layout(showlegend=True)
fig = go.Figure(data=data,layout=layout)

py.iplot(fig, filename='default-legend')

<b>Answer:</b>

<b>d.  HKS difference (5 points).</b>  For a point $x_0\in S$, plot the function $\|\mathrm{HKS}(x_0)-\mathrm{HKS}(x)\|_2$ as a function of $x\in S$, where $x_0$ is a vertex on the hand of the human model.  Is the HKS discriminative?

In [8]:
x0 = hand1 # source vertex

# COMPUTE FUNCTION HERE
result = np.zeros((nv,1))
    
textured_mesh(mesh,result,'hksdiff')

<b>Text answer:</b>

<hr/>
<b>Extra credit (10 points; 20 for truly exceptional work)</b>

<p>Every 6.838 assignment can be accompanied with <i>open-ended</i> extra credit.  Excited about an algorithm we covered in class?  Implement it in here!  Curious about whether ideas from shape analysis can be applied to a particular field?  Give it a try.  Write it up below, or share a second document and tell us where to find it.</p>